In [2]:
from arcgis.gis import GIS

In [5]:
url = 'https://adventure.maps.arcgis.com'
arcgis_client_id = "2MonB9785a7oRJEl"

itm_id = '6f237db9d0404c329bbb2de485f19b3c'

In [6]:
gis = GIS(url, client_id=arcgis_client_id)

gis

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://adventure.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=2MonB9785a7oRJEl&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=t3ivD6j5R5RzWJPBPjci7l3XP3ezNA&allow_verification=false


Enter code obtained on signing in using SAML:  ········


GIS @ https://adventure.maps.arcgis.com version:2025.1

In [9]:
itm = gis.content.get(itm_id)

lyr = itm.layers[0]

In [12]:
fs = lyr.query()

fs

<FeatureSet> 1 features

In [13]:
fs.to_dict()

{'features': [{'geometry': {'paths': [[[-12870856.1737675, 6158471.65250669],
      [-12834474.5369609, 6083592.02867284]]],
    'spatialReference': {'wkid': 102100, 'latestWkid': 3857}},
   'attributes': {'OBJECTID': 1,
    'Shape__Length': 83250.11448776473,
    'name': 'test',
    'river_name': 'test river'}}],
 'objectIdFieldName': 'OBJECTID',
 'globalIdFieldName': '',
 'spatialReference': {'wkid': 102100, 'latestWkid': 3857},
 'geometryType': 'esriGeometryPolyline',
 'fields': [{'name': 'OBJECTID',
   'type': 'esriFieldTypeOID',
   'alias': 'OBJECTID',
   'sqlType': 'sqlTypeOther',
   'domain': None,
   'defaultValue': None},
  {'name': 'Shape__Length',
   'type': 'esriFieldTypeDouble',
   'alias': 'Shape__Length',
   'sqlType': 'sqlTypeDouble',
   'domain': None,
   'defaultValue': None},
  {'name': 'name',
   'type': 'esriFieldTypeString',
   'alias': 'Name',
   'sqlType': 'sqlTypeOther',
   'length': 256,
   'domain': None,
   'defaultValue': None},
  {'name': 'river_name',
   